<a href="https://colab.research.google.com/github/ykitaguchi77/Instance_segmentation/blob/main/MPLD_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MPLD analysis**

In [1]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
detect_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/295+cerebhq1-20000_yolo11l.pt"
seg_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/eyelid_caruncle_yolo11seg_1-139.pt"
obb_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/yolo11n_obb_1-295_1to139.pt"

In [ ]:
image_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images/Control_adult/1000.jpg"

In [ ]:
!pip install ultralytics --q

In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from google.colab import drive

# Google Driveをマウント (Colabで実行する場合)
# try:
#     drive.mount('/content/drive')
# except OSError as e:
#     if "already mounted" in str(e).lower():
#         print("Google Drive is already mounted.")
#     else:
#         raise e

# --- 1. パス設定 ---
detect_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/295+cerebhq1-20000_yolo11l.pt"
# seg_model_path と obb_model_path はこのスクリプトでは使用しません
# seg_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/eyelid_caruncle_yolo11seg_1-139.pt"
# obb_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/yolo11n_obb_1-295_1to139.pt"
image_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/Segmentation_test_images/Control_adult/1000.jpg"

# --- 2. モデルのロード ---
try:
    detect_model = YOLO(detect_model_path)
    print(f"Detection model '{detect_model_path}' loaded successfully.")
except Exception as e:
    print(f"Error loading detection model: {e}")
    # スクリプトの実行をここで停止するか、適切に処理
    exit()

# --- 3. 画像のロード ---
try:
    img_pil = Image.open(image_path).convert("RGB")
    img_cv_rgb = np.array(img_pil) # OpenCVで処理するためにNumpy配列 (RGB) に変換
    original_img_height, original_img_width = img_cv_rgb.shape[:2]
    print(f"Image '{image_path}' loaded successfully. Dimensions: {original_img_width}x{original_img_height}")
except FileNotFoundError:
    print(f"Error: Image file not found at '{image_path}'")
    exit()
except Exception as e:
    print(f"Error loading image: {e}")
    exit()

# --- 4. 物体検出の実行 ---
print("Running detection model...")
results = detect_model(img_pil, verbose=False) # verbose=Falseで検出ログを抑制
print("Detection complete.")

# --- 5. BBoxの処理と画像の切り抜き ---
cropped_images_pil = []

if results and len(results) > 0 and results[0].boxes:
    boxes = results[0].boxes.xyxy.cpu().numpy()  # バウンディングボックスをxyxy形式で取得
    print(f"Found {len(boxes)} bounding box(es).")

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)

        bbox_width = x2 - x1
        bbox_height = y2 - y1
        bbox_center_x = x1 + bbox_width / 2.0
        bbox_center_y = y1 + bbox_height / 2.0

        print(f"\nProcessing BBox {i+1}: [x1={x1}, y1={y1}, x2={x2}, y2={y2}]")
        print(f"  Original BBox width: {bbox_width}, height: {bbox_height}, center: ({bbox_center_x:.2f}, {bbox_center_y:.2f})")

        # 横幅を25%ずつ水増し (元の1.5倍)
        new_width = int(round(bbox_width * 1.5))
        # 縦幅は新しい横幅と一致させる
        new_height = new_width

        print(f"  Target new width: {new_width}, new height: {new_height}")

        # 切り抜き用の座標を計算 (中心を維持)
        crop_x1 = int(round(bbox_center_x - new_width / 2.0))
        crop_y1 = int(round(bbox_center_y - new_height / 2.0))
        crop_x2 = crop_x1 + new_width
        crop_y2 = crop_y1 + new_height

        print(f"  Calculated crop window: [x1'={crop_x1}, y1'={crop_y1}, x2'={crop_x2}, y2'={crop_y2}]")

        # パディング量の計算
        pad_left = max(0, -crop_x1)
        pad_top = max(0, -crop_y1)
        pad_right = max(0, crop_x2 - original_img_width)
        pad_bottom = max(0, crop_y2 - original_img_height)

        print(f"  Padding needed: top={pad_top}, bottom={pad_bottom}, left={pad_left}, right={pad_right}")

        # 実際に画像から切り取る範囲を調整 (画像範囲内に収める)
        actual_crop_x1_img = crop_x1 + pad_left
        actual_crop_y1_img = crop_y1 + pad_top
        actual_crop_x2_img = crop_x2 - pad_right
        actual_crop_y2_img = crop_y2 - pad_bottom

        print(f"  Actual crop from image: [x_start={actual_crop_x1_img}, y_start={actual_crop_y1_img}, x_end={actual_crop_x2_img}, y_end={actual_crop_y2_img}]")

        # 画像を切り抜く (Numpy配列スライス)
        # スライスする前に、座標が画像の範囲内にあることを確認
        actual_crop_x1_img = max(0, actual_crop_x1_img)
        actual_crop_y1_img = max(0, actual_crop_y1_img)
        actual_crop_x2_img = min(original_img_width, actual_crop_x2_img)
        actual_crop_y2_img = min(original_img_height, actual_crop_y2_img)

        if actual_crop_x1_img >= actual_crop_x2_img or actual_crop_y1_img >= actual_crop_y2_img:
            print(f"  Skipping BBox {i+1} due to invalid crop dimensions after clamping.")
            # 有効な切り抜き領域がない場合は、黒一色の画像を生成するか、スキップ
            # ここでは、指定サイズの黒画像を生成
            cropped_part = np.zeros((new_height, new_width, 3), dtype=np.uint8)
        else:
            cropped_part_from_img = img_cv_rgb[actual_crop_y1_img:actual_crop_y2_img, actual_crop_x1_img:actual_crop_x2_img]
            # パディングを適用
            # cv2.copyMakeBorderのvalueはBGRだが、黒(0,0,0)なのでRGBでも同じ
            cropped_part = cv2.copyMakeBorder(
                cropped_part_from_img,
                pad_top,
                pad_bottom,
                pad_left,
                pad_right,
                cv2.BORDER_CONSTANT,
                value=[0, 0, 0]  # 黒でパディング
            )

        # 稀にパディング計算や丸め誤差で1pxずれることがある場合、リサイズで最終サイズを保証
        if cropped_part.shape[0] != new_height or cropped_part.shape[1] != new_width:
            print(f"  Warning: Padded crop size ({cropped_part.shape[1]}x{cropped_part.shape[0]}) differs from target ({new_width}x{new_height}). Resizing.")
            cropped_part = cv2.resize(cropped_part, (new_width, new_height), interpolation=cv2.INTER_AREA)


        cropped_images_pil.append(Image.fromarray(cropped_part)) # Pillow Imageとしてリストに追加
        print(f"  Cropped and padded image {i+1} generated. Final size: {cropped_part.shape[1]}x{cropped_part.shape[0]}")

else:
    print("No objects detected or no bounding boxes found in the results.")

# --- 6. 切り抜いた画像の表示 ---
if cropped_images_pil:
    num_cropped = len(cropped_images_pil)
    # 表示する画像の数に応じて subplot のレイアウトを調整
    cols = min(num_cropped, 3) # 最大3列で表示
    rows = (num_cropped + cols - 1) // cols # 必要な行数

    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    axes = np.array(axes).ravel() # axesを1次元配列に変換して扱いやすくする

    for i, cropped_img_pil in enumerate(cropped_images_pil):
        axes[i].imshow(cropped_img_pil)
        axes[i].set_title(f"Cropped Image {i+1}")
        axes[i].axis("off")

    # 余分な subplot を非表示にする
    for j in range(num_cropped, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()
    print(f"\nDisplayed {num_cropped} cropped image(s).")
elif not (results and len(results) > 0 and results[0].boxes):
    pass # すでに "No objects detected..." のメッセージが表示されている
else:
    print("No valid bounding boxes were processed to generate cropped images.")


# --- 7. 追加モデルのロード ---
seg_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/eyelid_caruncle_yolo11seg_1-139.pt"
obb_model_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/models/yolo11n_obb_1-295_1to139.pt"

print("\n--- Loading additional models ---")
try:
    seg_model = YOLO(seg_model_path)
    print(f"Segmentation model '{seg_model_path}' loaded successfully.")
    obb_model = YOLO(obb_model_path)
    print(f"OBB model '{obb_model_path}' loaded successfully.")
except Exception as e:
    print(f"Error loading additional models: {e}")
    # エラーが発生した場合、以降の処理をスキップするかどうかを決定
    # ここでは、エラーメッセージを表示して終了する代わりに、merged_images_pilが空のまま処理を進める
    seg_model = None
    obb_model = None


# --- 8. 各切り抜き画像への推論と結果のマージ ---
merged_images_pil = []

if not cropped_images_pil:
    print("No cropped images from the previous step to process.")
elif seg_model is None or obb_model is None:
    print("One or more additional models failed to load. Skipping merge step.")
else:
    print(f"\n--- Processing {len(cropped_images_pil)} cropped image(s) with segmentation and OBB models ---")
    for i, base_cropped_pil in enumerate(cropped_images_pil):
        print(f"  Processing cropped image {i+1}/{len(cropped_images_pil)}...")

        # 元の切り抜き画像 (Pillow RGB) をコピーして作業用にする
        # 推論は元のクリーンな画像で行う
        current_pil_image = base_cropped_pil.copy()
        current_rgb_numpy = np.array(current_pil_image) # RGB NumPy array

        # 1. セグメンテーション推論と描画
        # plot()はBGRのNumPy配列を返す。入力imgはRGB/BGRのNumPy配列を受け付ける。
        # ここでは、まずセグメンテーションの結果を元のRGB画像に描画する。
        # plot()のimgに渡す配列はコピーする方が安全。
        img_with_seg_bgr = None
        try:
            print(f"    Running segmentation model on image {i+1} with retina_masks=True...")
            results_seg = seg_model(
                current_pil_image,
                verbose=False,
                conf=0.35, # confはお好みで調整
                retina_masks=True # ★ 高品質マスク生成オプションを追加
            )
            if results_seg and results_seg[0].masks is not None: # マスクが存在する場合のみ描画
                img_with_seg_bgr = results_seg[0].plot(
                    img=current_rgb_numpy.copy(), # 元のRGB画像に描画
                    pil=False,                    # NumPy配列(BGR)で結果取得
                    masks=True,                   # マスクを描画
                    boxes=False,                  # セグメンテーションのBBoxは描画しない (任意)
                    labels=True                   # ラベルを描画
                )
                print(f"    Segmentation applied to image {i+1}.")
            else:
                print(f"    No segmentation masks found by seg_model for image {i+1}.")
                # マスクがない場合は、元の画像をBGRに変換したものを使用
                img_with_seg_bgr = cv2.cvtColor(current_rgb_numpy.copy(), cv2.COLOR_RGB2BGR)

        except Exception as e:
            print(f"    Error during segmentation on image {i+1}: {e}")
            # エラー発生時は、元の画像をBGRに変換したものを次のステップのベースとする
            img_with_seg_bgr = cv2.cvtColor(current_rgb_numpy.copy(), cv2.COLOR_RGB2BGR)


        # 2. OBB推論と描画
        # OBBモデルの推論は、元のクリーンな画像 (current_pil_image) で行う。
        # 描画は、セグメンテーションが描画された画像 (img_with_seg_bgr) に対して行う。
        final_img_np_bgr = None
        try:
            print(f"    Running OBB model on image {i+1}...")
            results_obb = obb_model(current_pil_image, verbose=False, conf=0.35) # confはお好みで調整
            if results_obb and hasattr(results_obb[0], 'obb') and results_obb[0].obb is not None: # OBBが存在する場合のみ描画
                 # OBBの結果を、セグメンテーションが描画済みの画像 (img_with_seg_bgr) に重ねて描画
                final_img_np_bgr = results_obb[0].plot(
                    img=img_with_seg_bgr.copy(), # セグメンテーション描画済みのBGR画像
                    pil=False,                 # NumPy配列(BGR)で結果取得
                    masks=False,               # OBBモデルは通常マスク出力しない
                    boxes=True,                # OBBを描画 (これがOBBを描画するはず)
                    labels=True                # ラベルを描画
                )
                print(f"    OBB applied to image {i+1}.")
            else:
                print(f"    No OBB detections found by obb_model for image {i+1}.")
                final_img_np_bgr = img_with_seg_bgr.copy() # OBBがない場合は直前の画像

        except Exception as e:
            print(f"    Error during OBB detection on image {i+1}: {e}")
            # エラー発生時は、セグメンテーション描画後の画像を最終結果とする
            final_img_np_bgr = img_with_seg_bgr.copy()

        # 最終結果 (BGR NumPy array) をPillow Image (RGB) に変換してリストに追加
        final_img_pil = Image.fromarray(cv2.cvtColor(final_img_np_bgr, cv2.COLOR_BGR2RGB))
        merged_images_pil.append(final_img_pil)
        print(f"  Finished processing and merging for image {i+1}.")

# --- 9. マージされた画像の表示 ---
if merged_images_pil:
    print(f"\n--- Displaying {len(merged_images_pil)} merged image(s) ---")
    num_merged = len(merged_images_pil)
    # 表示する画像の数に応じて subplot のレイアウトを調整
    cols = min(num_merged, 3) # 最大3列で表示
    rows = (num_merged + cols - 1) // cols # 必要な行数

    fig, axes = plt.subplots(rows, cols, figsize=(7 * cols, 7 * rows)) # figsizeを少し調整
    if num_merged == 1: # 画像が1枚の場合、axesはオブジェクトであり、配列ではない
        axes = np.array([axes])
    axes = axes.ravel() # axesを1次元配列に変換して扱いやすくする

    for i, merged_img_pil in enumerate(merged_images_pil):
        axes[i].imshow(merged_img_pil)
        axes[i].set_title(f"Merged Result {i+1}")
        axes[i].axis("off")

    # 余分な subplot があれば非表示にする
    for j in range(num_merged, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()
else:
    if cropped_images_pil and (seg_model is not None and obb_model is not None):
        print("No images were successfully merged. Check logs for errors.")
    # cropped_images_pilが空だった場合やモデルロード失敗のメッセージは既に表示されている

In [ ]:
!pip install japanize-matplotlib --q
import japanize_matplotlib # インポートするだけで日本語対応が改善されます

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import math
import os
import torch

# --- 定数定義 ---
OBB_CLASS_PUPIL = 'Pupil'
OBB_CLASS_IRIS = 'Iris'
SEG_CLASS_EYELID = 'Eyelid'
CORNEAL_DIAMETER_MM = 12.0
RADIAL_ANGLES_DEG = [0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180]

# --- 出力ディレクトリ作成 ---
output_dir = "/content/mrmpd_results"
os.makedirs(output_dir, exist_ok=True)
print(f"結果は '{output_dir}' ディレクトリに保存されます。")

# --- ヘルパー関数: OBBデータ抽出 (変更なし) ---
def get_obb_data_by_class_name(obb_results_obj, target_class_name, model_obb_names, img_idx_for_debug="N/A"):
    if obb_results_obj is None or not hasattr(obb_results_obj, 'obb') or obb_results_obj.obb is None or \
       not hasattr(obb_results_obj.obb, 'cls') or len(obb_results_obj.obb.cls) == 0:
        return None
    target_cls_indices = []
    if isinstance(model_obb_names, dict):
        for cls_id, name in model_obb_names.items():
            if name == target_class_name:
                target_cls_indices.append(int(cls_id))
    elif isinstance(model_obb_names, list):
        for cls_id, name in enumerate(model_obb_names):
            if name == target_class_name:
                target_cls_indices.append(cls_id)
    if not target_cls_indices: return None
    best_obb_data = None
    highest_conf = -1.0
    if not hasattr(obb_results_obj.obb, 'conf') or not hasattr(obb_results_obj.obb, 'xywhr'):
        return None
    detected_classes = obb_results_obj.obb.cls.cpu().numpy().astype(int)
    confidences = obb_results_obj.obb.conf.cpu().numpy()
    xywhr_data = obb_results_obj.obb.xywhr.cpu().numpy()
    found_instances = 0
    for i in range(len(detected_classes)):
        if detected_classes[i] in target_cls_indices:
            found_instances +=1
            current_conf = confidences[i]
            if current_conf > highest_conf:
                highest_conf = current_conf
                best_obb_data = {
                    "center_x": xywhr_data[i][0], "center_y": xywhr_data[i][1],
                    "width": xywhr_data[i][2], "height": xywhr_data[i][3],
                    "angle_rad": xywhr_data[i][4], "confidence": current_conf
                }
    return best_obb_data if found_instances > 0 and best_obb_data is not None else None

# --- ヘルパー関数: セグメンテーションマスク抽出 (変更なし) ---
def get_mask_by_class_name(seg_results_obj, target_class_name, model_seg_names, target_shape_hw, img_idx_for_debug="N/A"):
    if seg_results_obj.masks is None: return None
    if not hasattr(seg_results_obj.masks, 'data') or seg_results_obj.masks.data is None: return None
    if seg_results_obj.boxes is None: return None
    target_cls_indices = []
    if isinstance(model_seg_names, dict):
        for cls_id, name in model_seg_names.items():
            if name == target_class_name: target_cls_indices.append(int(cls_id))
    elif isinstance(model_seg_names, list):
        for cls_id, name in enumerate(model_seg_names):
            if name == target_class_name: target_cls_indices.append(cls_id)
    if not target_cls_indices: return None
    relevant_masks_data = []
    all_detected_cls_indices = seg_results_obj.boxes.cls.cpu().numpy().astype(int)
    raw_masks_tensor = seg_results_obj.masks.data
    if raw_masks_tensor is None or len(raw_masks_tensor) == 0: return None
    if len(all_detected_cls_indices) != raw_masks_tensor.shape[0]:
        print(f"WARNING (Image {img_idx_for_debug}, get_mask): Mismatch boxes ({len(all_detected_cls_indices)}) and masks ({raw_masks_tensor.shape[0]}).")
    num_instances_of_target_class_found = 0
    for i in range(len(all_detected_cls_indices)):
        if i >= raw_masks_tensor.shape[0]: break
        current_model_cls_idx = all_detected_cls_indices[i]
        if current_model_cls_idx in target_cls_indices:
            num_instances_of_target_class_found += 1
            try:
                mask_proto = raw_masks_tensor[i]
                mask_proto_tensor = mask_proto.unsqueeze(0).unsqueeze(0)
                if mask_proto_tensor.shape[2:] == target_shape_hw:
                    upsampled_mask_tensor = mask_proto_tensor.squeeze()
                else:
                    upsampled_mask_tensor = torch.nn.functional.interpolate(
                        mask_proto_tensor, size=target_shape_hw, mode='bilinear', align_corners=False
                    ).squeeze()
                mask_np = upsampled_mask_tensor.cpu().numpy()
                mask_np_binary = (mask_np > 0.5).astype(np.uint8) * 255
                contours, _ = cv2.findContours(mask_np_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                area = sum(cv2.contourArea(c) for c in contours) if contours else 0
                if area > 0: relevant_masks_data.append({'mask': mask_np_binary, 'area': area})
            except Exception as e:
                print(f"ERROR (Image {img_idx_for_debug}, get_mask): Exception processing mask for instance {i} of '{target_class_name}': {e}")
                continue
    if not relevant_masks_data: return None
    largest_mask_data = max(relevant_masks_data, key=lambda x: x['area'])
    return largest_mask_data['mask']

# --- MRMPD解析メイン処理 ---
print("\n--- MRMPD解析を開始します ---")

if 'cropped_images_pil' not in locals() or not cropped_images_pil:
    print("重大なエラー: `cropped_images_pil` が見つからないか空です。")
    exit("エラー: 入力画像リスト `cropped_images_pil` が定義されていません。")
if 'obb_model' not in locals() or not hasattr(obb_model, 'names'):
    print("重大なエラー: `obb_model` がロードされていないか、`names` 属性がありません。")
    exit("エラー: `obb_model` が定義されていません。")
if 'seg_model' not in locals() or not hasattr(seg_model, 'names'):
    print("重大なエラー: `seg_model` がロードされていないか、`names` 属性がありません。")
    exit("エラー: `seg_model` が定義されていません。")

all_mrmpd_results_data = []

for img_idx, pil_image_clean_crop in enumerate(cropped_images_pil):
    base_filename = getattr(pil_image_clean_crop, 'filename', f"image_{img_idx + 1}")
    base_filename = os.path.splitext(os.path.basename(base_filename))[0]
    debug_img_id_str = f"{base_filename}_idx{img_idx + 1}"

    print(f"\n--- 画像 {debug_img_id_str} のMRMPD解析を開始 ---")
    current_mrmpd_data = {"image_id": debug_img_id_str, "status": "処理開始"}

    img_np_rgb = np.array(pil_image_clean_crop.convert("RGB"))
    img_h, img_w = img_np_rgb.shape[:2]

    pupil_center = None
    pixels_per_mm = None

    # 1. OBBモデルによる瞳孔と虹彩の情報取得 (変更なし)
    try:
        obb_results_list = obb_model(pil_image_clean_crop, verbose=False, conf=0.25)
        if not obb_results_list: raise ValueError("OBBモデルが結果を返しませんでした。")
        obb_result_obj = obb_results_list[0]

        pupil_obb_data = get_obb_data_by_class_name(obb_result_obj, OBB_CLASS_PUPIL, obb_model.names, img_idx_for_debug=debug_img_id_str)
        if not pupil_obb_data: raise ValueError(f"'{OBB_CLASS_PUPIL}' OBBが見つかりません。")
        pupil_center = (int(round(pupil_obb_data["center_x"])), int(round(pupil_obb_data["center_y"])))
        current_mrmpd_data["pupil_center_px"] = pupil_center
        print(f"  瞳孔中心 (OBBより): {pupil_center}")

        iris_obb_data = get_obb_data_by_class_name(obb_result_obj, OBB_CLASS_IRIS, obb_model.names, img_idx_for_debug=debug_img_id_str)
        if not iris_obb_data: raise ValueError(f"'{OBB_CLASS_IRIS}' OBBが見つかりません。")
        major_axis_pixels = max(iris_obb_data["width"], iris_obb_data["height"])
        if not major_axis_pixels > 0: raise ValueError(f"'{OBB_CLASS_IRIS}' OBBの長径が0です。")
        pixels_per_mm = major_axis_pixels / CORNEAL_DIAMETER_MM
        current_mrmpd_data["pixels_per_mm"] = pixels_per_mm
        current_mrmpd_data["iris_major_axis_px"] = major_axis_pixels
        print(f"  虹彩の長径 (OBBより): {major_axis_pixels:.2f}px。計算されたスケール: {pixels_per_mm:.2f} px/mm")

    except Exception as e:
        error_msg = f"OBBモデル処理中にエラー: {e}"
        print(f"エラー (画像 {debug_img_id_str}): {error_msg}")
        current_mrmpd_data.update({"error": error_msg, "status": "OBBエラー"})
        all_mrmpd_results_data.append(current_mrmpd_data)
        continue

    # 2. セグメンテーションモデルによる 'Eyelid' マスク取得 (変更なし)
    eyelid_mask_area = None
    try:
        seg_results_list = seg_model(pil_image_clean_crop, verbose=False, conf=0.25, retina_masks=True)
        if not seg_results_list: raise ValueError("セグメンテーションモデルが結果を返しませんでした。")
        seg_result_obj = seg_results_list[0]
        eyelid_mask_area = get_mask_by_class_name(seg_result_obj, SEG_CLASS_EYELID, seg_model.names, (img_h, img_w), img_idx_for_debug=debug_img_id_str)
        if eyelid_mask_area is None: raise ValueError(f"'{SEG_CLASS_EYELID}' マスクが見つかりません。")
        print(f"  '{SEG_CLASS_EYELID}' マスク取得成功。")
    except Exception as e:
        error_msg = f"セグメンテーションモデル処理中にエラー: {e}"
        print(f"エラー (画像 {debug_img_id_str}): {error_msg}")
        current_mrmpd_data.update({"error": error_msg, "status": "Segエラー"})
        all_mrmpd_results_data.append(current_mrmpd_data)
        continue

    # 3. 放射線と 'Eyelid' 領域マスクとの交点計算 (★ロジック変更箇所★)
    mpld_mm_values = {}
    intersection_points_coords = {}
    max_search_radius = int(math.sqrt(img_w**2 + img_h**2))
    pupil_cx, pupil_cy = pupil_center

    # 瞳孔中心がマスク内にあるか最初に確認
    if not (0 <= pupil_cx < img_w and 0 <= pupil_cy < img_h and eyelid_mask_area[pupil_cy, pupil_cx] > 0):
        print(f"警告 (画像 {debug_img_id_str}): 瞳孔中心 ({pupil_center}) がEyelidマスクの範囲外、またはマスク値が0です。MPLD計算は不正確になる可能性があります。")
        # この場合でも処理を続行するが、結果は期待できないかもしれない

    print(f"  MPLD計算中 (瞳孔中心: {pupil_center}):")
    for angle_deg in RADIAL_ANGLES_DEG:
        angle_rad = math.radians(angle_deg)

        last_known_mask_point = None # マスク領域内だった最後の点を記録
        current_distance_at_last_mask_point = np.nan

        found_exit_point_for_this_angle = False

        for r_step in range(1, max_search_radius + 1): # 瞳孔中心の隣(r=1)から探索開始
            x_curr = int(round(pupil_cx + r_step * math.cos(angle_rad)))
            y_curr = int(round(pupil_cy - r_step * math.sin(angle_rad)))

            if not (0 <= x_curr < img_w and 0 <= y_curr < img_h): # 現在の探索点が画像範囲外
                # 画像範囲外に出る直前までが探索範囲。
                # last_known_mask_point が None でなければ、それが縁。
                if last_known_mask_point is not None:
                    mpld_mm_values[angle_deg] = current_distance_at_last_mask_point / pixels_per_mm
                    intersection_points_coords[angle_deg] = last_known_mask_point
                    found_exit_point_for_this_angle = True
                break # この方向の探索は終了

            if eyelid_mask_area[y_curr, x_curr] > 0: # 現在点がマスク内
                last_known_mask_point = (x_curr, y_curr)
                current_distance_at_last_mask_point = r_step # 瞳孔中心からの直線距離(ピクセル)
            else: # 現在点がマスク外 (eyelid_mask_area[y_curr, x_curr] == 0)
                  # 直前の last_known_mask_point が「出る点」だったことになる
                if last_known_mask_point is not None: # 瞳孔中心から少なくとも1ピクセルはマスク内を通った場合
                    mpld_mm_values[angle_deg] = current_distance_at_last_mask_point / pixels_per_mm
                    intersection_points_coords[angle_deg] = last_known_mask_point
                    found_exit_point_for_this_angle = True
                # else: 瞳孔中心のすぐ隣(r=1)が既にマスク外。この場合、last_known_mask_pointはNoneのまま。距離は0またはNaN。
                #       この場合は、found_exit_point_for_this_angle は False のまま。
                break # この方向の探索は終了

        if not found_exit_point_for_this_angle:
            # ループを完走しても出る点が見つからなかった (画像端までマスク内だったか、そもそもマスクに入らなかった)
            if last_known_mask_point is not None: # 画像端までマスク内だった
                mpld_mm_values[angle_deg] = current_distance_at_last_mask_point / pixels_per_mm
                intersection_points_coords[angle_deg] = last_known_mask_point
            else: # 瞳孔中心の隣からずっとマスク外だった場合など
                mpld_mm_values[angle_deg] = np.nan
                intersection_points_coords[angle_deg] = None

    current_mrmpd_data["mpld_mm"] = mpld_mm_values
    current_mrmpd_data["intersections_px"] = intersection_points_coords
    current_mrmpd_data["radial_angles_deg"] = RADIAL_ANGLES_DEG
    print(f"  MPLD計算完了。")

    # 4. 結果の描画、極座標プロット、非対称比の計算 (変更なし)
    img_viz_mrmpd_pil = pil_image_clean_crop.copy().convert("RGB")
    draw = ImageDraw.Draw(img_viz_mrmpd_pil)
    draw.ellipse((pupil_cx-3, pupil_cy-3, pupil_cx+3, pupil_cy+3), fill="red", outline="red")
    for angle_deg, int_pt in intersection_points_coords.items():
        if int_pt:
            draw.line([(pupil_cx, pupil_cy), int_pt], fill="yellow", width=2)
            draw.ellipse((int_pt[0]-3, int_pt[1]-3, int_pt[0]+3, int_pt[1]+3), fill="cyan", outline="cyan")
        else:
            angle_rad = math.radians(angle_deg)
            end_x_no_int = int(round(pupil_cx + 30 * math.cos(angle_rad)))
            end_y_no_int = int(round(pupil_cy - 30 * math.sin(angle_rad)))
            draw.line([(pupil_cx, pupil_cy), (end_x_no_int, end_y_no_int)], fill=(128,128,128,180), width=1)

    plt.figure(figsize=(8, 8))
    plt.imshow(img_viz_mrmpd_pil)
    plt.title(f"MRMPD解析結果 - 画像 {debug_img_id_str}\n瞳孔中心: {pupil_center}, スケール: {pixels_per_mm:.2f} px/mm")
    plt.axis("off")
    viz_filename = os.path.join(output_dir, f"mrmpd_visualization_{debug_img_id_str}.png")
    try: img_viz_mrmpd_pil.save(viz_filename); print(f"  可視化画像を保存しました: {viz_filename}")
    except Exception as e: print(f"  可視化画像の保存に失敗しました: {e}")
    plt.show()

    valid_plot_angles_rad = []
    valid_plot_radii_mm = []
    for angle_deg_plot in RADIAL_ANGLES_DEG:
        val_mm = mpld_mm_values.get(angle_deg_plot, np.nan)
        if not np.isnan(val_mm):
            valid_plot_angles_rad.append(math.radians(angle_deg_plot))
            valid_plot_radii_mm.append(val_mm)

    if valid_plot_radii_mm:
        fig_polar, ax_polar = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(7, 7))
        ax_polar.plot(valid_plot_angles_rad, valid_plot_radii_mm, marker='o', linestyle='-')
        ax_polar.set_theta_zero_location("E"); ax_polar.set_theta_direction(1)
        ax_polar.set_xticks(np.deg2rad(RADIAL_ANGLES_DEG))
        ax_polar.set_xticklabels([f"{angle}°" for angle in RADIAL_ANGLES_DEG])
        max_r_val = max(valid_plot_radii_mm) if valid_plot_radii_mm else 5.0
        ax_polar.set_rmax(math.ceil(max_r_val * 1.1 if max_r_val > 0 else 5.0))
        ax_polar.set_title(f"MPLDの極座標プロット (mm) - 画像 {debug_img_id_str}", va='bottom', pad=20)
        polar_plot_filename = os.path.join(output_dir, f"mrmpd_polar_plot_{debug_img_id_str}.png")
        try: fig_polar.savefig(polar_plot_filename); print(f"  極座標プロットを保存しました: {polar_plot_filename}")
        except Exception as e: print(f"  極座標プロットの保存に失敗しました: {e}")
        plt.show()
    else:
        print(f"  画像 {debug_img_id_str} の極座標プロットを作成するための有効なMPLDデータがありません。")

    asymmetry_ratios_calculated = {}
    ratio_angle_pairs = [(105, 75), (120, 60), (135, 45), (150, 30), (165, 15), (180, 0)]
    print("  左右非対称比 (側頭側 / 鼻側):")
    for t_angle, n_angle in ratio_angle_pairs:
        temporal_val = mpld_mm_values.get(t_angle, np.nan)
        nasal_val = mpld_mm_values.get(n_angle, np.nan)
        ratio_label = f"{t_angle}°/{n_angle}°"
        if not np.isnan(temporal_val) and not np.isnan(nasal_val) and nasal_val != 0:
            ratio = temporal_val / nasal_val
            asymmetry_ratios_calculated[ratio_label] = ratio
            print(f"    {ratio_label}: {ratio:.2f}")
        else:
            asymmetry_ratios_calculated[ratio_label] = np.nan
            print(f"    {ratio_label}: N/A (データ不足)")
    current_mrmpd_data["asymmetry_ratios"] = asymmetry_ratios_calculated
    current_mrmpd_data["status"] = "処理完了"
    all_mrmpd_results_data.append(current_mrmpd_data)

print("\n--- MRMPD解析 全画像処理完了 ---")